In [3]:
# NOTICE: I marked the places you should experiment with: # CHANGE, 
# so it should be easier to find the places to change and try different methods. 

# comment out these lines if you have already installed these modules
#!pip install nltk
#!pip install tensorflow
#!pip install keras

import sklearn as sk
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
import string
import nltk
import tensorflow
from sklearn.model_selection import train_test_split
from collections import Counter
from pandas import DataFrame
from matplotlib import pyplot
import re
from nltk.stem import WordNetLemmatizer 
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sriku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sriku\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Preprocess Code

In [5]:
def process_news(news):
    _news = news.replace('b\"', "")
    _news = _news.replace('b\'', "")
    _news = _news.lower()
    _news = re.sub("[^a-zA-Z]", " ",_news)
    _news = re.sub('[\s]+', ' ', _news)
    
    tokens = _news.split(" ")
    if "" in tokens:
        tokens.remove("")
    
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    #remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    
    _news = ' '.join(tokens)    
     
    return _news

Load data

In [6]:
def read_data():

    data = pd.read_csv("C:/Users/sriku/Downloads/CSC503project-master/CSC503project-master/Datasets/djia/Combined_News_DJIA.csv")
    
    dfs = []
    data["News"] = ""
    for i in range(1,25):
        col = "Top"+str(i)
        data["News"] = data["News"] +" "+ data[col]
    data = data.dropna()
    data['PreProcessedNews'] = data['News'].map(process_news)
    
    data = data[['Date', 'News', 'PreProcessedNews', 'Label']]
    
    stock_prices = "C:/Users/sriku/Downloads/CSC503project-master/CSC503project-master/Datasets/djia/upload_DJIA_table.csv"
    stock_data = pd.read_csv(stock_prices)
    
    print(data.head(2))
    print(stock_data.head(2))
    
    
    #merged_dataframe = data.merge(stock_data, how='inner', on='Date')
    merged_dataframe = pd.merge(data, stock_data, how='inner', on = 'Date')
    from sklearn.utils import shuffle
    for i in range(5):
        merged_dataframe = shuffle(merged_dataframe)

    Xy_train = merged_dataframe[:int(len(data)*0.8)]
    Xy_test = merged_dataframe[int(len(data)*0.8):]
    print(Xy_train.shape)
    print(Xy_test.shape)
    return merged_dataframe, Xy_train, Xy_test

Load data function call

In [7]:
news, Xy_train, Xy_test = read_data()
news.head(5)

         Date                                               News  \
0  2008-08-08   b"Georgia 'downs two Russian warplanes' as co...   
1  2008-08-11   b'Why wont America and Nato help us? If they ...   

                                    PreProcessedNews  Label  
0  georgia two russian warplane country move brin...      0  
1  wont america nato help wont help help iraq bus...      1  
         Date          Open          High           Low         Close  \
0  2016-07-01  17924.240234  18002.380859  17916.910156  17949.369141   
1  2016-06-30  17712.759766  17930.609375  17711.800781  17929.990234   

      Volume     Adj Close  
0   82160000  17949.369141  
1  133030000  17929.990234  
(1588, 10)
(398, 10)


,Date,News,PreProcessedNews,Label,Open,High,Low,Close,Volume,Adj Close
1118,2013-01-23,$20 trillion oil basin discovered in Australi...,trillion oil basin discovered australia set tu...,1,13712.209961,13794.290039,13710.129883,13779.330078,104490000,13779.330078
1458,2014-05-30,India arrests police officers over gang rape ...,india arrest police officer gang rape siberian...,1,16697.330078,16721.220703,16648.849609,16717.169922,105190000,16717.169922
374,2010-02-05,"b"" \t\n\nUganda's 'kill gays' friends' bill: ...",nuganda kill gay friend bill nanti gay law wor...,1,10003.690430,10031.959961,9835.089844,10012.230469,308320000,10012.230469
313,2009-11-05,"b'We broke the law, admits CIA agent convicte...",broke law admits cia agent convicted rendition...,1,9807.799805,10013.070312,9807.799805,10005.959961,211040000,10005.959961
113,2009-01-21,b'Huh? The NYTimes publishes an Op-Ed by Liby...,huh nytimes publishes op ed libyan leader muam...,1,7949.169922,8243.549805,7936.189941,8228.099609,410040000,8228.099609


In [8]:
X_train = Xy_train['PreProcessedNews']
X_test = Xy_test['PreProcessedNews']
y_train = Xy_train['Label'].to_numpy()
y_test = Xy_test['Label'].to_numpy()

Prepare data for vectorization

In [9]:
def prepare_data(train_docs, test_docs, mode):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode testing data set
    Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
    
    return Xtrain, Xtest


Create vectors for each sentence
* binary
* count
* tfidf
* freq

In [10]:
#modes = ['binary', 'count', 'tfidf', 'freq']

# CHANGE MODE
mode = 'tfidf'

X_train, X_test = prepare_data(X_train, X_test, mode)

In [11]:
X_test.shape

(398, 25781)

In [12]:
X_train.shape

(1588, 25781)

In [13]:
X_train

array([[0.        , 0.00371747, 0.00371747, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00384615, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.02295082, 0.00327869, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0078125 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00743494, 0.01115242, ..., 0.00371747, 0.00371747,
        0.00371747]])

Reduce the dimentionality since this is a sparse vector

In [14]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

# Make sure to comment one or the other.

# CHANGE TO SVD
# CHANGE: the n_components - Reduce dimensionality
svd = TruncatedSVD(n_components=40, n_iter=30)
X_train = svd.fit_transform(X_train)
X_test = svd.transform(X_test)

# # CHANGE TO PCA
# # CHANGE: the the n_components - Reduce dimensionality
# pca = PCA(n_components=2)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)


In [15]:
X_train.shape

(1588, 40)

In [16]:
X_test.shape

(398, 40)

Apply machine learning example

In [17]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [18]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

# Please refer to:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
# for more information of which parameters to change for each kernel

# CHANGE: Set the parameters by cross-validation
tuned_parameters = [
#{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],'C': [0.00001,0.0001,0.001,0.01,0.1,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]},

# {'kernel': ['sigmoid'], 'gamma': [1e-0,1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9],'C': [0.00001,0.0001,0.001,0.01,0.1,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]},

# {'kernel': ['poly'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.00001,0.0001,0.001,0.01,0.1,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20], 
# 'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
 
{'kernel': ['linear'], 'C': [0.00001,0.0001,0.001,0.01,0.1,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]}

]

scores = ['accuracy', 'precision']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    
    if score == "accuracy":
    
        clf = GridSearchCV(
            SVC(), tuned_parameters, scoring=score
        )
    
    if score == "precision":
        clf = GridSearchCV(
            SVC(), tuned_parameters, scoring='%s_macro' % score
        )
    
    
    clf.fit(X_train, y_train)


    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true,y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

# Tuning hyper-parameters for accuracy

Best parameters set found on development set:

{'C': 1e-05, 'kernel': 'linear'}

Grid scores on development set:

0.536 (+/-0.002) for {'C': 1e-05, 'kernel': 'linear'}
0.536 (+/-0.002) for {'C': 0.0001, 'kernel': 'linear'}
0.536 (+/-0.002) for {'C': 0.001, 'kernel': 'linear'}
0.517 (+/-0.037) for {'C': 0.01, 'kernel': 'linear'}
0.503 (+/-0.022) for {'C': 0.1, 'kernel': 'linear'}
0.499 (+/-0.018) for {'C': 1, 'kernel': 'linear'}
0.502 (+/-0.017) for {'C': 2, 'kernel': 'linear'}
0.503 (+/-0.019) for {'C': 3, 'kernel': 'linear'}
0.502 (+/-0.020) for {'C': 4, 'kernel': 'linear'}
0.502 (+/-0.020) for {'C': 5, 'kernel': 'linear'}
0.502 (+/-0.020) for {'C': 6, 'kernel': 'linear'}
0.501 (+/-0.016) for {'C': 7, 'kernel': 'linear'}
0.501 (+/-0.022) for {'C': 8, 'kernel': 'linear'}
0.502 (+/-0.021) for {'C': 9, 'kernel': 'linear'}
0.503 (+/-0.019) for {'C': 10, 'kernel': 'linear'}
0.502 (+/-0.020) for {'C': 20, 'kernel': 'linear'}

Detailed classification r

C:\Users\sriku\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sriku\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sriku\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sriku\anaconda3\lib\site-packages\sklearn\metr

Best parameters set found on development set:

{'C': 0.01, 'kernel': 'linear'}

Grid scores on development set:

0.268 (+/-0.001) for {'C': 1e-05, 'kernel': 'linear'}
0.268 (+/-0.001) for {'C': 0.0001, 'kernel': 'linear'}
0.268 (+/-0.001) for {'C': 0.001, 'kernel': 'linear'}
0.492 (+/-0.054) for {'C': 0.01, 'kernel': 'linear'}
0.487 (+/-0.029) for {'C': 0.1, 'kernel': 'linear'}
0.483 (+/-0.024) for {'C': 1, 'kernel': 'linear'}
0.486 (+/-0.024) for {'C': 2, 'kernel': 'linear'}
0.486 (+/-0.026) for {'C': 3, 'kernel': 'linear'}
0.485 (+/-0.028) for {'C': 4, 'kernel': 'linear'}
0.485 (+/-0.028) for {'C': 5, 'kernel': 'linear'}
0.485 (+/-0.028) for {'C': 6, 'kernel': 'linear'}
0.484 (+/-0.024) for {'C': 7, 'kernel': 'linear'}
0.485 (+/-0.030) for {'C': 8, 'kernel': 'linear'}
0.486 (+/-0.028) for {'C': 9, 'kernel': 'linear'}
0.486 (+/-0.026) for {'C': 10, 'kernel': 'linear'}
0.485 (+/-0.028) for {'C': 20, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full 

In [ ]:
# {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [50, 100, 150, 200]},

# {'kernel': ['sigmoid'], 'gamma': [1e-0,1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9],'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50]},

# {'kernel': ['poly'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5], 'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20], 
# 'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
 
# {'kernel': ['linear'], 'C': [50, 100, 150, 200]
    
# {'kernel': ['rbf'], 'gamma': [1e-2,1e-1, 1,1e+1,1e+2],'C': np.logspace(-2,2,5)},

# {'kernel': ['sigmoid'], 'gamma': np.logspace(-2,2,5),'C': np.logspace(-2,2,5)},

# #{'kernel': ['poly'], 'gamma': np.logspace(-2,2,5), 'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20], 
# #'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
 
# {'kernel': ['linear'], 'C': np.logspace(-2,2,5)

In [18]:
##### OLD RESULTS - BEFORE SHUFFLING ######
##### P.S : No shuffling, so results will just be the same anytime your run the code #######
# mode = binary

# training first 80%, testing last 20%, n_components = 20 : {'C': 0.001, 'kernel': 'linear'} - 0.51
# training last 20%, testing first 80% : {'C': 1e-05, 'kernel': 'linear'} - 0.52 
# training first 70%, testing last 30% : {'C': 1e-05, 'kernel': 'linear'} - 0.53
# training first 60%, testing last 40% : {'C': 1e-05, 'kernel': 'linear'} - 0.53

# 70/30 {'C': 0.01, 'degree': 7, 'gamma': 0.1, 'kernel': 'poly'} - 0.53

# 50/50 sigmoid - 0.54

# mode = tfidf
# 50/50 {'C': 1e-05, 'kernel': 'linear'} - 0.54

In [ ]:
##### NEW RESULTS - UPON SHUFFLING ######
##### P.S : Because we are shuffling the data, the below results will keep changing #####
# shuffle merged dataframe = 5 times
# svc : n_components = 40, iter = 30

# mode = binary
# 80/20 , {'C': 1e-05, 'kernel': 'linear'} - 0.53 accuracy
# 80/20 , {'C': 4, 'kernel': 'linear'} - 0.54 precision

# mode = count
# 80/20 , {'C': 1e-05, 'kernel': 'linear'} - 0.50

# mode = tfidf
# 80/20, {'C': 1e-05, 'kernel': 'linear'} - 0.55 accuracy
# 80/20, {'C': 0.01, 'kernel': 'linear'} - 0.56 precision